### Classical Delta_l calculation

The alogorithm generates a random lattice configuration respecting the null total charge condition.

The lattice is then associated with a reduced lattice with the same Deltal value.

Finally the Deltal of both the reduced and original lattice is computed.

In [1]:
from qiskit import *
from qiskit import IBMQ, Aer

from qiskit.circuit import ParameterVector
from qiskit.visualization.bloch import Bloch
from qiskit.algorithms.optimizers import SPSA
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Gate
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.tools.visualization import plot_histogram
from scipy.optimize import curve_fit
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA #linear algebra
from copy import deepcopy
import math
import time

In [2]:
def generator(n,p):
    '''
    n - number of fermions/antifermions
    p - number of filled sites

    Generates a 2n vector with p filled fermions and p filled antifermions
    '''

    vec = np.zeros(2*n)
    #print(vec)

    fermions = np.random.choice(range(n),p, replace=False)
    #print(fermions)

    antifermions = np.random.choice(range(n),p,replace=False)
    #print(antifermions)

    for i in range(p): 
        vec[2*fermions[i]]=1
        vec[2*antifermions[i]+1]=1
    return vec

In [3]:
def Dealtal_calculation (v):

    '''
    given an occupancy configuration v 
    where even sites are reserved to fermions 
    and odd sites to antifermions,
    produces the associated Deltal
    
    '''

    #PRINCIPAL VARIABLES DEFINITION
    Deltal = 0

    #AUXILIARY VARIABLES
    l = [0]
    n = len(v)

    for i in range(0, n):
        l.append(l[i]+ v[i]*(-1)**i)

    Deltal = max(l)-min(l)

    return Deltal

In [4]:
def Reduction(v):

    '''
    this function associates to a given lattice configuration
    a new configuration, with the same Deltal value,
      with a number of sites <= to the original one
    
    '''

    #PRINCIPAL VARIABLES DEFINITION
    V = []
    W = []
    v_new = []

    #AUXILIARY VARIABLES
    n = len(v)
    zeros_counter = 0
    ones_counter = 0

    #v TRANSLATION

    for i in range(0,n):
        if i == n:
            break
        if v[i] == 1:
            break
        i = i+1

    if i == n:
        print('THE VECTOR v IS NULL!')
        W=[0,0]
    else:
        for j in range(0, n): 
                v_new.append(v[(j+i)%n])

        v = v_new

        
        #ZEROES REDUCTION

        for i in range(0, n):
            if v[i] == 0:
                zeros_counter = zeros_counter + 1 
            else:
                if zeros_counter % 2 == 0:
                    V.append(1)
                else:
                    V.append(0)
                    V.append(1)
                zeros_counter = 0

        if zeros_counter % 2 == 1:
            V.append(0)

        #V TRANSLATION
        N = len(V)
        v_new = []
        k = 0

        for k in range(len(V)):
            if k == N:
                break
            if V[k] == 0:
                break
            k = k+1

        if k == N:
            print('THE VECTOR V IS COMPLETELY FILLED!')
            W=[1,1]
        else:
            for j in range(0, N): 
                    v_new.append(V[(j+i)%N])

            V = v_new
            #ONES REDUCTION


            for i in range(0, N):
                if V[i] == 1:
                    ones_counter = ones_counter + 1 
                else:
                    if ones_counter % 2 == 0:
                        if ones_counter != 0:
                            W.append(1)
                            W.append(1)
                        W.append(0)
                    else:
                        W.append(1)
                        W.append(0)
                    ones_counter = 0

            if ones_counter != 0:
                if ones_counter % 2 == 0:
                    W.append(1)
                    W.append(1)
                else:
                    W.append(1)

    return W

In [5]:
n=10 #number of lattice sites

for i in range(1):
    v = generator(n,5)
    print(v)
    print(Dealtal_calculation(v))
    w=Reduction(v)
    print(w)
    print(Dealtal_calculation(w))
    print('------------------------')


[0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0.]
3.0
[1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
3
------------------------
